#Discretization Tutorial

To complete the definition of the numerical model, we add numerics to the physics described by the domain, equation, and initial/boundary conditions. There are many options for numerics including choices  such as the time discretization, spatial approximation space, quadrature rules, numerical stabilization schemes, numerical fluxes, and numerical linear algebra.

In [2]:
from proteus import default_p as p
from proteus import default_n as n
import proteus as pr
n.timeIntegration = pr.TimeIntegration.BackwardEuler_cfl
n.stepController = pr.StepControl.Min_dt_cfl_controller
n.runCFL=0.99
n.femSpaces = {0:pr.FemTools.C0_AffineLinearOnSimplexWithNodalBasis}
n.elementQuadrature = pr.Quadrature.SimplexGaussQuadrature(p.nd,3)
n.elementBoundaryQuadrature = pr.Quadrature.SimplexGaussQuadrature(p.nd-1,3)
n.subgridError = pr.SubgridError.AdvectionDiffusionReaction_ASGS(p.coefficients,
                                                                 p.nd,lag=False)
n.shockCapturing = pr.ShockCapturing.ResGradQuad_SC(p.coefficients,
                                                            p.nd,
                                                            shockCapturingFactor=0.75,
                                                            lag=True)
n.numericalFluxType = pr.NumericalFlux.Advection_DiagonalUpwind_Diffusion_SIPG_exterior
n.nnx=41; n.nny=41
n.tnList=[float(i)/40.0 for i in range(11)]
n.parallelPartitioningType = pr.MeshTools.MeshParallelPartitioningTypes.element
n.nLayersOfOverlapForParallel = 1

AttributeError: 'NoneType' object has no attribute 'nc'

#Nonlinear Iteration

In [ ]:
n.multilevelNonlinearSolver = pr.NonlinearSolvers.Newton
n.maxLineSearches=0
#convergence criteria
n.maxNonlinearIts=50
n.tolFac = 0.0
n.nl_atol_res=1.0e-4


# Linear Iteration

In [ ]:
n.matrix = pr.LinearAlgebraTools.SparseMatrix
n.multilevelLinearSolver = pr.LinearSolvers.KSP_petsc4py
# PETSc solver configuration
from petsc4py import PETSc
OptDB = PETSc.Options()
OptDB.setValue("ksp_type", "gmres")
OptDB.setValue("pc_type", "jacobi")
# convergence criteria
n.linearSolverConvergenceTest = 'r-true'
n.linTolFac = 0.001
n.l_atol_res = 0.001*n.nl_atol_res